In [ ]:
# FAJAR ARIF KURNIAWAN
# BOGOR

In [ ]:
import pandas as pd
df = pd.read_csv('bbc-news-data.csv', sep='\t')

df.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [ ]:
category = pd.get_dummies(df.category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='category')
df_baru

,filename,title,content,business,entertainment,politics,sport,tech
0,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,1,0,0,0,0
1,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,1,0,0,0,0
2,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,1,0,0,0,0
3,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,1,0,0,0,0
4,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2220,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,0,0,0,0,1
2221,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,0,0,0,0,1
2222,399.txt,Be careful how you code,A new European directive could put software w...,0,0,0,0,1
2223,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...,0,0,0,0,1


In [ ]:
df_new = df_baru.drop(columns=['filename'])
df_new

,title,content,business,entertainment,politics,sport,tech
0,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,1,0,0,0,0
1,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,1,0,0,0,0
2,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,1,0,0,0,0
3,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,1,0,0,0,0
4,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,1,0,0,0,0
...,...,...,...,...,...,...,...
2220,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,0,0,0,0,1
2221,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,0,0,0,0,1
2222,Be careful how you code,A new European directive could put software w...,0,0,0,0,1
2223,US cyber security chief resigns,The man making sure US computer networks are ...,0,0,0,0,1


In [ ]:
import nltk, os, re, string

from keras.layers import Input, LSTM, Bidirectional, SpatialDropout1D, Dropout, Flatten, Dense, Embedding, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

nltk.download('wordnet')
nltk.download('stopwords') 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
comment = df_baru['title'].values + '' + df_new['content'].values
label = df_new[['business', 'entertainment', 'politics', 'sport', 'tech']].values
label

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split
comment_latih, comment_test, label_latih, label_test = train_test_split(comment, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x', filters='!"#$%&()*+,-./:;<=>@[\]^_`{|}~ ')
tokenizer.fit_on_texts(comment_latih)
tokenizer.fit_on_texts(comment_test)

sekuens_latih = tokenizer.texts_to_sequences(comment_latih)
sekuens_test = tokenizer.texts_to_sequences(comment_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers
model = tf.keras.Sequential([
     tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
     tf.keras.layers.LSTM(128),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dropout(0.5),
     tf.keras.layers.Dense(64, activation='relu'),
     tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          320000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               98816     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 443,909
Trainable params: 443,909
Non-tr

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9 and logs.get('val_accuracy')>0.85):
      self.model.stop_training = True
      print("\nThe accuracy of the training set and the validation set has reached > 90%!")
callbacks = myCallback()

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, batch_size=128, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2, callbacks=[callbacks])

Epoch 1/30
14/14 - 255s - loss: 1.6066 - accuracy: 0.2247 - val_loss: 1.6042 - val_accuracy: 0.2067 - 255s/epoch - 18s/step
Epoch 2/30
14/14 - 253s - loss: 1.5937 - accuracy: 0.2489 - val_loss: 1.5806 - val_accuracy: 0.2090 - 253s/epoch - 18s/step
Epoch 3/30
14/14 - 247s - loss: 1.5743 - accuracy: 0.3326 - val_loss: 1.5706 - val_accuracy: 0.4000 - 247s/epoch - 18s/step
Epoch 4/30
14/14 - 246s - loss: 1.5064 - accuracy: 0.5011 - val_loss: 1.3318 - val_accuracy: 0.5056 - 246s/epoch - 18s/step
Epoch 5/30
14/14 - 245s - loss: 1.2236 - accuracy: 0.5781 - val_loss: 1.1459 - val_accuracy: 0.5528 - 245s/epoch - 18s/step
Epoch 6/30
14/14 - 245s - loss: 0.8265 - accuracy: 0.6551 - val_loss: 0.7922 - val_accuracy: 0.6629 - 245s/epoch - 18s/step
Epoch 7/30
14/14 - 243s - loss: 0.6081 - accuracy: 0.7691 - val_loss: 0.8662 - val_accuracy: 0.6944 - 243s/epoch - 17s/step
Epoch 8/30
14/14 - 248s - loss: 0.3879 - accuracy: 0.8888 - val_loss: 0.6347 - val_accuracy: 0.7865 - 248s/epoch - 18s/step
Epoch 9/